# Loading training data

In [8]:
import csv
import numpy as np
X = []
with open("./data/train/crawler/data/tweets.txt.text", newline='', encoding='utf8') as file_data:
    i = 0 
    for row in file_data:
        if i < 5000:
            i = i + 1
            X.append(row)
        else:
            break

y = []
with open("./data/train/crawler/data/tweets.txt.labels", newline='', encoding='utf8') as file_data:
    j = 0
    for row in file_data:
        if j < 5000:
            j = j + 1
            y.append(row.replace("\n",""))
        else:
            break


In [9]:
X = np.array(X)
y = np.array(y)

# Data vectorisation

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

# Data test loading and vectorisation

In [11]:
test=[]
with open("./data/test/us_test.text", newline='', encoding='utf8') as test_data:
    file = test_data.readlines()
    for row in file:
        test.append(row.replace("\n",""))

test = np.asarray(test)
test_dtm = vect.transform(test)

In [12]:
test_label = []
with open("./data/test/us_test.labels", newline='', encoding='utf8') as test_data_label:
    file = test_data_label.readlines()
    for row in file:
        test_label.append(row.replace("\n",""))
test_label = np.asarray(test_label)
test_label = test_label.reshape(-1,1)

# Model implementation and training
## Measurements calculation and results

In [13]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, f1_score, jaccard_score, classification_report

models = [
    MLPClassifier(solver = 'adam', activation = 'logistic', learning_rate = 'adaptive')
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    model.fit(X_dtm, y)
    predict_label = model.predict(test_dtm)
    acc = accuracy_score(predict_label,test_label)
    f1 = f1_score(predict_label, test_label, average = 'weighted')
    cm = multilabel_confusion_matrix(test_label,predict_label)
    jaccard = jaccard_score(test_label, predict_label, average='micro')
    print(classification_report(test_label, predict_label))
    print ("la matrice de confusion : ")
    print(cm)
    entries.append((model_name, acc, f1, jaccard))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'accuracy', 'f1', 'jaccard'])

/home/julien/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


              precision    recall  f1-score   support

           0       0.28      0.50      0.36     10798
           1       0.16      0.19      0.17      4830
          10       0.12      0.07      0.08      1432
          11       0.42      0.28      0.33      1949
          12       0.26      0.19      0.22      1265
          13       0.06      0.01      0.02      1114
          14       0.05      0.02      0.03      1306
          15       0.13      0.10      0.12      1244
          16       0.06      0.02      0.03      1153
          17       0.60      0.48      0.53      1545
          18       0.18      0.05      0.07      2417
          19       0.03      0.01      0.01      1010
           2       0.24      0.31      0.27      4534
           3       0.10      0.08      0.09      2605
           4       0.44      0.23      0.30      3716
           5       0.05      0.07      0.06      1613
           6       0.10      0.07      0.08      1996
           7       0.19    

In [14]:
print (cv_df)

      model_name  accuracy        f1   jaccard
0  MLPClassifier   0.22776  0.249888  0.128515


# Grid Search use to find the best parameters for this model

In [17]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

# Adam works better ; Logistic works better
parameters = {'solver':('lbfgs', 'adam'), 'activation':('relu', 'logistic'), 
              'alpha':[0.0001, 0.00001, 0.001]
               }

mlp = MLPClassifier(learning_rate = 'adaptive')

clf = GridSearchCV(mlp, parameters, cv=3, n_jobs=-1)

clf.fit(X_dtm, y)
                            
print(clf.cv_results_)